In [1]:
import numpy as np
import pandas as pd
from os import getcwd

from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,MinMaxScaler


getcwd()

'C:\\Users\\Alec\\Documents\\Programming_Resources\\Boot_Camp\\19_supervised_machine_learning\\homework'

# Load data

In [2]:
train_df = pd.read_csv('2019loans.csv')
test_df = pd.read_csv('2020Q1loans.csv')

In [3]:
train_df.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,57107,57107,13375.0,0.1797,483.34,MORTGAGE,223000.0,Not Verified,low_risk,n,...,100.0,50.0,0.0,0.0,577150.0,122018.0,32000.0,170200.0,N,N
1,141451,141451,21000.0,0.1308,478.68,MORTGAGE,123000.0,Source Verified,low_risk,n,...,85.0,33.3,0.0,0.0,132750.0,27896.0,15900.0,35398.0,N,N
2,321143,321143,20000.0,0.1240,448.95,MORTGAGE,197000.0,Source Verified,low_risk,n,...,85.7,33.3,0.0,0.0,628160.0,114043.0,22600.0,90340.0,N,N
3,11778,11778,3000.0,0.1240,100.22,RENT,45000.0,Not Verified,low_risk,n,...,100.0,16.7,1.0,0.0,42006.0,20761.0,19900.0,15406.0,N,N
4,169382,169382,30000.0,0.1612,1056.49,MORTGAGE,133000.0,Source Verified,low_risk,n,...,100.0,66.7,0.0,0.0,283248.0,109056.0,79500.0,58778.0,N,N


In [4]:
test_df.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,67991,67991,40000.0,0.0819,814.70,MORTGAGE,140000.0,Not Verified,low_risk,n,...,97.7,0.0,0.0,0.0,527975.0,70914.0,74600.0,99475.0,N,N
1,25429,25429,6000.0,0.1524,208.70,RENT,55000.0,Not Verified,low_risk,n,...,66.7,0.0,0.0,0.0,34628.0,23460.0,5900.0,23628.0,N,N
2,38496,38496,3600.0,0.1695,128.27,RENT,42000.0,Not Verified,low_risk,n,...,100.0,0.0,0.0,0.0,23100.0,19183.0,7300.0,15000.0,N,N
3,19667,19667,20000.0,0.1524,478.33,RENT,100000.0,Not Verified,low_risk,n,...,100.0,50.0,0.0,0.0,56481.0,43817.0,13800.0,35981.0,N,N
4,37505,37505,3600.0,0.1240,120.27,RENT,50000.0,Not Verified,low_risk,n,...,100.0,25.0,0.0,0.0,45977.0,32448.0,21000.0,24977.0,N,N


# Data Processing

In [5]:
# Convert categorical data to numeric and separate target feature for training data 

X_train = train_df.drop(columns='loan_status')
y_train = train_df['loan_status']

X_train_dummies = pd.get_dummies(X_train,drop_first=True)


In [6]:
# Convert categorical data to numeric and separate target feature for testing data 

X_test = test_df.drop(columns='loan_status')
y_test = test_df['loan_status']

X_test_dummies = pd.get_dummies(X_test,drop_first=True)


In [7]:
# compare columns in Dataframes
print(f'number of Test_dummies columns -> {len(X_test_dummies.columns)} ')
print(f'number of Train_dummies columns -> {len(X_train_dummies.columns)} ')

number of Test_dummies columns -> 86 
number of Train_dummies columns -> 87 


Insert missing columns to preserve formatting?

In [8]:
# add missing dummy variables to testing set 

missing_col = []
for col in X_train_dummies.columns:
    if col not in X_test_dummies:
        missing_col.append(col)
        X_test_dummies[col] = 0
missing_col

['debt_settlement_flag_Y']

-------------

# Logistic Regression - Unscaled Data

In [9]:
# Train the Logistic Regression model on the unscaled data and print the model score 
model = LogisticRegression(solver='sag',max_iter=4000)
model.fit(X_train_dummies,y_train)

C:\Users\Alec\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogisticRegression(max_iter=4000, solver='sag')

In [10]:
print(f'Train Score -> {model.score(X_train_dummies,y_train)}')
print(f'Test Score -> {model.score(X_test_dummies,y_test)}')

Train Score -> 0.653776683087028
Test Score -> 0.5263717566992769


# Random Forest - Unscaled Data

Tends to be mroe accurate but there is a higher danger of overfitting

-------------

In [11]:
# Train a Random Forest Classifier model and print the model score 
clf = RandomForestClassifier(random_state=12, n_estimators=500).fit(X_train_dummies,y_train)

In [12]:
print(f'Training Score -> {clf.score(X_train_dummies, y_train)}')
print(f'Testing Score -> {clf.score(X_test_dummies, y_test)}')

Training Score -> 1.0
Testing Score -> 0.6171841769459804


-------------

# Scale the Data

Observe the differences in scores after testing over scaled data

In [13]:
scaler = StandardScaler().fit(X_train_dummies)
X_train_scaled = scaler.transform(X_train_dummies)
X_test_scaled = scaler.transform(X_test_dummies)

In [14]:
scaler = MinMaxScaler().fit(X_train_dummies)
X_train_Min_Max = scaler.transform(X_train_dummies)
X_test_Min_Max = scaler.transform(X_test_dummies)

-------------

# Logistic Regression - Scaled Data

### Standard Scaler

In [15]:
# Train the Logistic Regression model on the scaled data and print the model score 

model = LogisticRegression(solver='sag',max_iter=4000)
model.fit(X_train_scaled,y_train)

LogisticRegression(max_iter=4000, solver='sag')

-------------

### Min Max Scaler

In [16]:
min_max_model = model.fit(X_train_Min_Max,y_train)

-------------

In [17]:
print(f'Scaled Train Score -> {model.score(X_train_scaled,y_train)}')
print(f'Scaled Test Score -> {model.score(X_test_scaled,y_test)}')
print(f'Min_Max Scaled Test Score -> {min_max_model.score(X_test_Min_Max,y_test)}')
print(f'Unscaled Test Score -> {model.score(X_test_dummies,y_test)}')


Scaled Train Score -> 0.6961412151067323
Scaled Test Score -> 0.6607826456826882
Min_Max Scaled Test Score -> 0.5952786048490004
Unscaled Test Score -> 0.5535942152275627


-------------

# Random Forest - Scaled Data

### Standard Scaler

In [18]:
# Train a Random Forest Classifier model on the scaled data and print the model score

clf = RandomForestClassifier(random_state=12, n_estimators=500).fit(X_train_scaled,y_train)

-------------

### Min Max Scaler

In [19]:
Min_Max_clf = RandomForestClassifier(random_state=12, n_estimators=500).fit(X_train_Min_Max,y_train)

In [20]:
print(f'Scaled Training Score -> {clf.score(X_train_scaled, y_train)}')
print(f'Scaled Testing Score -> {clf.score(X_test_scaled, y_test)}')
print(f'Min_Max Scaled Test Score -> {Min_Max_clf.score(X_test_Min_Max,y_test)}')


Scaled Training Score -> 1.0
Scaled Testing Score -> 0.6169715014887281
Min_Max Scaled Test Score -> 0.6178222033177372


-------------

#### N_estimators increased to 750

-------------

In [21]:
# Train a Random Forest Classifier model on the scaled data and print the model score

clf = RandomForestClassifier(random_state=12, n_estimators=750).fit(X_train_scaled,y_train)

In [22]:
Min_Max_clf = RandomForestClassifier(random_state=12, n_estimators=750).fit(X_train_Min_Max,y_train)

In [23]:
print(f'Scaled Training Score -> {clf.score(X_train_scaled, y_train)}')
print(f'Scaled Testing Score -> {clf.score(X_test_scaled, y_test)}')
print(f'Min_Max Scaled Test Score -> {Min_Max_clf.score(X_test_Min_Max,y_test)}')

Scaled Training Score -> 1.0
Scaled Testing Score -> 0.6135686941726924
Min_Max Scaled Test Score -> 0.6152700978307103


-------------